In [19]:
import pandas as pd
from pathlib import Path
import pandas as pd

base_path = Path.cwd()

Reading the original OPV file with all PDFs and raw data:

In [20]:
opv_path = base_path.parent / "data" / "opv_export_masked_20220901.csv"

df = pd.read_csv(opv_path)

In [21]:
df.head()

,pdf_filename,opv_filename,maskedid,aedob_shift,aeexamdate_shift,eye,trial_rx_sphere,trial_rx_cylinder,trial_rx_axis,pupil_diameter,...,n3_i21_pdp,t3_i21_pdp,t9_i21_pdp,t15_i21_pdp,n9_i27_pdp,n3_i27_pdp,t3_i27_pdp,t9_i27_pdp,test_type,vatype
0,1.2.276.0.75.2.5.80.25.3.190123133623332.93180...,1.2.276.0.75.2.5.80.25.3.190123133622780.93180...,VIP009279,11/17/1949,12/24/2016,L,3.25,0.0,0,NaN,...,< 0.5%,Not Significant,Not Significant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.2.276.0.75.2.5.80.25.3.190123133624307.93180...,1.2.276.0.75.2.5.80.25.3.190123133623936.93180...,VIP009279,11/17/1949,12/24/2016,R,3.25,0.0,0,3.9,...,< 0.5%,< 1%,< 0.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.2.276.0.75.2.5.80.25.3.190123133722362.93180...,1.2.276.0.75.2.5.80.25.3.190123133721903.93180...,VIP112677,06/01/1962,03/04/2011,L,2.00,0.0,0,4.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.2.276.0.75.2.5.80.25.3.190123133723983.93180...,1.2.276.0.75.2.5.80.25.3.190123133723701.93180...,VIP112677,06/01/1962,11/20/2010,L,1.25,0.0,0,4.4,...,< 0.5%,< 0.5%,< 0.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.2.276.0.75.2.5.80.25.3.190123133724566.93180...,1.2.276.0.75.2.5.80.25.3.190123133724289.93180...,VIP112677,06/01/1962,11/20/2010,R,2.00,0.0,0,4.7,...,Not Significant,< 0.5%,< 5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Calculate age in years from aedob_shift and aeexamdate_shift:
df['age'] = (pd.to_datetime(df['aeexamdate_shift']) - pd.to_datetime(df['aedob_shift'])).dt.days // 365

In [23]:
# Keep only testpattern == "24-2"
df = df[df['testpattern'] == "24-2"]

In [24]:
# Keep only relevant columns:
df = df[['maskedid', 'eye', 'age', 'aedob_shift', 'aeexamdate_shift', 'pdf_filename', 'opv_filename']]
df.head()

,maskedid,eye,age,aedob_shift,aeexamdate_shift,pdf_filename,opv_filename
0,VIP009279,L,67,11/17/1949,12/24/2016,1.2.276.0.75.2.5.80.25.3.190123133623332.93180...,1.2.276.0.75.2.5.80.25.3.190123133622780.93180...
1,VIP009279,R,67,11/17/1949,12/24/2016,1.2.276.0.75.2.5.80.25.3.190123133624307.93180...,1.2.276.0.75.2.5.80.25.3.190123133623936.93180...
2,VIP112677,L,48,06/01/1962,03/04/2011,1.2.276.0.75.2.5.80.25.3.190123133722362.93180...,1.2.276.0.75.2.5.80.25.3.190123133721903.93180...
3,VIP112677,L,48,06/01/1962,11/20/2010,1.2.276.0.75.2.5.80.25.3.190123133723983.93180...,1.2.276.0.75.2.5.80.25.3.190123133723701.93180...
4,VIP112677,R,48,06/01/1962,11/20/2010,1.2.276.0.75.2.5.80.25.3.190123133724566.93180...,1.2.276.0.75.2.5.80.25.3.190123133724289.93180...


In [ ]:
# Create a counter for how many visual fields per patient, per eye, and also assign a number from 1 to n for each visual field/eye, according to date:
df['visual_field_count'] = df.groupby(['maskedid', 'eye']).cumcount() + 1

# Sort
df = df.sort_values(by=['maskedid', 'eye', 'aeexamdate_shift'])

# Sequencial number of tests per pacient/eye
df['visual_field_number'] = df.groupby(['maskedid', 'eye']).cumcount() + 1

# Total count per patient/eye
df['visual_field_count'] = df.groupby(['maskedid', 'eye'])['aeexamdate_shift'].transform('count')


In [27]:
df.head()

,maskedid,eye,age,aedob_shift,aeexamdate_shift,pdf_filename,opv_filename,visual_field_count,visual_field_number
3350,VIP000030,R,78,05/14/1938,10/13/2016,1.2.276.0.75.2.5.80.25.3.190125121048070.93180...,1.2.276.0.75.2.5.80.25.3.190125121046730.93180...,1,1
214,VIP000040,L,86,09/05/1925,12/01/2011,1.2.276.0.75.2.5.80.25.3.190123155921416.93180...,1.2.276.0.75.2.5.80.25.3.190123155920710.93180...,1,1
2393,VIP000040,R,86,09/05/1925,12/01/2011,1.2.276.0.75.2.5.80.25.3.190124124305206.93180...,1.2.276.0.75.2.5.80.25.3.190124154418396.93180...,1,1
1806,VIP000137,L,43,06/04/1970,03/21/2014,1.2.276.0.75.2.5.80.25.3.190124062348979.93180...,1.2.276.0.75.2.5.80.25.3.190125210944911.93180...,2,1
1259,VIP000137,L,47,06/04/1970,07/01/2017,1.2.276.0.75.2.5.80.25.3.190124003044673.93180...,1.2.276.0.75.2.5.80.25.3.190124003044211.93180...,2,2


In [28]:
df.to_csv(base_path.parent / "data" / "opv_24-2_prepared.csv", index=False)